Данный проект был создан студентами МИРЭА Калинином Александром и Чуфистовым Егором. Задача этого проекта: в телеграмме преобразовать аудиосообщение в текстовое, а затем его суммаризовать. Полученный результат отправить пользователю. Сообщения могут быть распознаны только на английском языке. Данный проект использовал такие модели, как whisper, pegasus, pipeline.
Поиском подходящих моделей и их использованием занимался Калинин Александр, а над созданием телеграмм-бота работал Чуфистов Егор. 

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install pydub
!pip install pyTelegramBotApi
!sudo apt update && sudo apt install ffmpeg

In [ ]:
from transformers import PegasusTokenizer, pipeline
import whisper
import pprint
import os
from datetime import datetime
from pydub import AudioSegment

import telebot
import uuid
from telebot import types

In [ ]:
#Создание токенизации у модели Pegasus
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)

In [ ]:
#Загрузка моделей
model = whisper.load_model("medium")
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer, framework="pt")

In [ ]:
TOKEN = "5415102077:AAGNWApJFAQf-CDghNGshz1bWyFMAzRJap0"

language = 'ru_RU'
bot = telebot.TeleBot(TOKEN)


@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(chat_id=message.chat.id, text="Для того, чтобы сделать перевод, "
                                                            "ответьте голосовым сообщением "
                                                            "на это сообщение.")


def recognize(file_path):
    result = model.transcribe(file_path)
    summary = summarizer(result["text"], min_length=15, max_length=250)
    return summary[0]["summary_text"]


@bot.message_handler(content_types=['voice'])
def voice_processing(message):
    filename = str(uuid.uuid4())
    file_name_full = f"/{filename}.ogg"
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open(file_name_full, 'wb') as new_file:
        new_file.write(downloaded_file)
    bot.send_message(chat_id=message.chat.id, text="Идет обработка аудиосообщения, "
                                                        "пожалуйста, подождите.")
    text = recognize(file_name_full)
    bot.reply_to(message, text)
    os.remove(file_name_full)


bot.polling()
